In [178]:
import numpy as np
import pandas as pd
from surprise import KNNWithMeans
from surprise import Dataset, Reader, SVD
from surprise.model_selection import GridSearchCV

In [179]:
df = pd.read_csv("user-item-100k-cleaned.csv")
df_raw = pd.read_csv("cleaned_top_5k.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110187 entries, 0 to 110186
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype
---  ------      --------------   -----
 0   Unnamed: 0  110187 non-null  int64
 1   user_id     110187 non-null  int64
 2   app_id      110187 non-null  int64
 3   rating      110187 non-null  int64
dtypes: int64(4)
memory usage: 3.4 MB


In [180]:
# Here is how df looks
df.head()

,Unnamed: 0,user_id,app_id,rating
0,0,1,538,1
1,1,1,137,1
2,2,1,3,5
3,3,1,0,2
4,4,1,4,4


In [181]:
# Here is how df_raw looks
df_raw.head()

,Unnamed: 0,App Name,App Id,Category,Rating,Rating Count,Installs,Minimum Installs,Maximum Installs,Free,...,Size,Minimum Android,Developer Website,Released,Last Updated,Content Rating,Privacy Policy,Ad Supported,In App Purchases,Editors Choice
0,385470,WhatsApp Messenger,com.whatsapp,Communication,4.0,138557570.0,"5,000,000,000+",5.000000e+09,6265637751,True,...,19.2,Varies with device,1,"Oct 18, 2010","Jun 09, 2021",Everyone,1,False,False,True
1,304824,Instagram,com.instagram.android,Social,3.8,120206190.0,"1,000,000,000+",1.000000e+09,3559871277,True,...,19.2,Varies with device,1,"Apr 3, 2012","Jun 14, 2021",Teen,1,True,True,True
2,2222701,Facebook,com.facebook.katana,Social,2.3,117850066.0,"5,000,000,000+",5.000000e+09,6782619635,True,...,19.2,Varies with device,1,"Jun 16, 2021","Jun 16, 2021",Teen,1,True,True,False
3,881403,YouTube,com.google.android.youtube,Video Players & Editors,4.4,112440547.0,"5,000,000,000+",5.000000e+09,9766230924,True,...,19.2,Varies with device,1,"Oct 20, 2010","Jun 16, 2021",Teen,1,True,False,False
4,244319,Garena Free Fire - Rampage,com.dts.freefireth,Action,4.2,89177097.0,"500,000,000+",5.000000e+08,976536041,True,...,19.2,4.1 and up,1,"Dec 7, 2017","Jun 04, 2021",Mature 17+,1,False,True,True


In [182]:
# transfer the data from pandas dataframe into surprise's dataset
reader = Reader(rating_scale=(1, 5))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(df[["user_id", "app_id", "rating"]], reader)

In [183]:
# perform cross validation to find the best parameters

param_grid = {
    "n_epochs": [5, 10],
    "lr_all": [0.002, 0.005],
    "reg_all": [0.4, 0.6]
}
gs = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], cv=3)

gs.fit(data)

print(gs.best_score["rmse"])
print(gs.best_params["rmse"])

1.2959981401160319
{'n_epochs': 5, 'lr_all': 0.002, 'reg_all': 0.6}


In [184]:
# create a model with the best parameters found in the previous step

svd = SVD(verbose=True, n_epochs=5, lr_all=0.002, reg_all=0.6)
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.2945  1.2951  1.2988  1.2961  0.0019  
MAE (testset)     1.1234  1.1250  1.1258  1.1247  0.0010  
Fit time          0.27    0.33    0.30    0.30    0.03    
Test time         0.23    0.21    0.35    0.26    0.06    


{'test_rmse': array([1.29445307, 1.29506386, 1.29875412]),
 'test_mae': array([1.12340798, 1.12498066, 1.12583445]),
 'fit_time': (0.2710907459259033, 0.33457493782043457, 0.2968447208404541),
 'test_time': (0.233048677444458, 0.21033287048339844, 0.3491828441619873)}

In [186]:
trainset = data.build_full_trainset()
svd.fit(trainset)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4


In [187]:
class AppRecommenderSystem:
    
    def __init__(self, ratings_dataset, app_metadata, model):
        self._ratings = ratings_dataset
        self._app_metadata = app_metadata
        self._model = model
        
    def _predict_rating(self, user_id, app_id):
        prediction = self._model.predict(uid=user_id, iid=app_id)
        return prediction.est
    
    def _get_app_metadata(self, app_id):
        return self._app_metadata.iloc[app_id, :]
        
    def _generate_recommendations(self, user_id, number_of_recommendations=5, threshold=3.7):
        appIds = np.arange(0, df_raw.shape[0])
        # shuffle the appIds to introduce some randomness
        random.shuffle(appIds)
        
        recommendations = []
        
        for i in range(0, len(appIds)):
            res = self._predict_rating(user_id, appIds[i])
            if(res > threshold):
                if appIds[i] not in recommendations:
                    recommendations.append(appIds[i])
                    if len(recommendations) == number_of_recommendations: 
                        return recommendations

    
        print("You need to lower the threshold...")
        return []
    
    def _generate_recommendations_dataframe(self, ids):
        apps = []
    
        for x in ids:
            apps.append(self._get_app_metadata(x))

        return pd.DataFrame(apps)
    
    def recommend(self, user_id, number_of_recommendations=5, threshold=3.7):
        result = self._generate_recommendations_dataframe(self._generate_recommendations(user_id, number_of_recommendations, threshold))
        
        return result
        
    def get_user_ratings(self, user_id):
        user_ratings = self._ratings[self._ratings['user_id'] == user_id]

        apps = []

        for x in user_ratings['app_id'].values:
            apps.append(self._app_metadata.iloc[x, 1])

        applications = pd.DataFrame(apps)
        applications["users_given_rating"] = user_ratings['rating'].values

        return applications
        
        

In [188]:
app_recommender = AppRecommenderSystem(df, df_raw, svd)

In [191]:
app_recommender.recommend(1, 2, 3.7)

,Unnamed: 0,App Name,App Id,Category,Rating,Rating Count,Installs,Minimum Installs,Maximum Installs,Free,...,Size,Minimum Android,Developer Website,Released,Last Updated,Content Rating,Privacy Policy,Ad Supported,In App Purchases,Editors Choice
3515,774291,Kitchen Craze: Free Cooking Games & kitchen Game,com.fme.KitchenCraze,Casual,4.4,234518.0,"10,000,000+",10000000.0,15165471,True,...,95.0,4.1 and up,1,"Dec 27, 2016","Jan 25, 2021",Everyone,1,True,True,True
983,1181705,MiChat - Free Chats & Meet New People,com.michatapp.im,Communication,4.3,865509.0,"50,000,000+",50000000.0,57386166,True,...,31.0,4.1 and up,1,"Jun 07, 2021","Jun 07, 2021",Mature 17+,1,True,False,False


In [190]:
app_recommender.get_user_ratings(1)

,0,users_given_rating
0,OLX,1
1,Temple Run,1
2,YouTube,5
3,WhatsApp Messenger,2
4,Garena Free Fire - Rampage,4
5,Facebook,4
6,8 Ball Pool,5
7,TikFans - Boost Followers and Likes for Tik Tok,5
8,eFootball PES 2021,4
9,Spotify: Listen to podcasts & find music you love,5
